In [157]:
import os
import pandas as pd

staging_path='Staging_Layer'
mart_path='Mart_Layer'
report_path='Report_Layer'

os.makedirs(staging_path, exist_ok=True)
os.makedirs(mart_path, exist_ok=True)
os.makedirs(report_path, exist_ok=True)

----------------------------------------------------------------------------------Staging Layer----------------------------------------------------------------------------------

In [133]:
import pandas as pd
employee = pd.read_csv('../Source/employee.csv')
course = pd.read_csv('../Source/courses.csv')
employee_course = pd.read_csv('../Source/employee_course.csv')

In [134]:
employee_df = pd.DataFrame(employee)
course_df = pd.DataFrame(course)
employee_course_df = pd.DataFrame(employee_course)

In [135]:
print(employee.info())
print(course.info())
print(employee_course_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                500 non-null    int64 
 1   empID             500 non-null    object
 2   name              500 non-null    object
 3   designation       500 non-null    object
 4   password          500 non-null    object
 5   email             500 non-null    object
 6   mobile            500 non-null    int64 
 7   performance_rate  500 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 31.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                50 non-null     int64 
 1   courseID          50 non-null     object
 2   name              50 non-null     object
 3   duration          50 non-null     object
 4

In [136]:
employee_df.to_csv('Staging_Layer/employee.csv', index=False)
course_df.to_csv('Staging_Layer/course.csv', index=False)
employee_course_df.to_csv('Staging_Layer/employee_course.csv',index=False)

---------------------------------------------------------------------------------Mart Layer---------------------------------------------------------------------------------

Course Table

In [137]:
course_mart_df = pd.read_csv('Staging_Layer/course.csv')
course_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                50 non-null     int64 
 1   courseID          50 non-null     object
 2   name              50 non-null     object
 3   duration          50 non-null     object
 4   difficulty_level  50 non-null     object
 5   learning_path     50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [138]:
course_mart_df.dropna(subset=['courseID'], inplace=True) # Drop rows with null courseID
course_mart_df['name'].fillna('Unknown', inplace=True)  # Filling with 'Unknown' for name
course_mart_df['duration'].fillna('Not Specified', inplace=True)  # Fill with 'Not Specified'
course_mart_df['difficulty_level'].fillna('Not Specified', inplace=True)  # Fill with 'Not Specified'
course_mart_df['learning_path'].fillna('Not Assigned', inplace=True)  # Fill with 'Not Assigned'
course_mart_df=course_mart_df.drop_duplicates() # Dropping duplicate values

C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\3279152316.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  course_mart_df['name'].fillna('Unknown', inplace=True)  # Filling with 'Unknown' for name
C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\3279152316.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 

In [139]:
def convert_duration_to_weeks(duration):
    if isinstance(duration, str):
        # Extract number and convert to weeks based on context
        if 'week' in duration.lower():
            return int(duration.split()[0])  # Returns the numeric part
        elif 'month' in duration.lower():
            return int(duration.split()[0]) * 4  # Assuming 1 month = 4 weeks
        elif 'year' in duration.lower():
            return int(duration.split()[0]) * 52 # Assuming 1 year = 52 weeks
    return None  # For 'Not Specified' or other values

# Apply conversion to a new column for numerical representation
course_mart_df['duration_weeks'] = course_mart_df['duration'].apply(convert_duration_to_weeks)

In [140]:
course_mart_df.to_csv('Mart_Layer/course.csv',index=False)
course_mart_df.head()

,id,courseID,name,duration,difficulty_level,learning_path,duration_weeks
0,1,COURSE001,Technical Writing,6 weeks,Advanced,Data Science,6
1,2,COURSE002,Database Management,2 weeks,Beginner,Cloud Computing,2
2,3,COURSE003,ITIL Foundation,5 weeks,Intermediate,Cloud Computing,5
3,4,COURSE004,Network Security,5 weeks,Beginner,Data Science,5
4,5,COURSE005,Machine Learning Basics,4 weeks,Advanced,Machine Learning,4


Employee_Course Table

In [141]:
employee_course_mart_df=pd.read_csv('Staging_Layer/employee_course.csv')
employee_course_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   empID            2000 non-null   object 
 1   courseID         2000 non-null   object 
 2   completion_rate  2000 non-null   float64
dtypes: float64(1), object(2)
memory usage: 47.0+ KB


In [142]:
employee_course_mart_df=employee_course_mart_df.drop_duplicates() # Dropping duplicate values

In [143]:
# Moving data into Mart Layer
employee_course_mart_df.to_csv('Mart_Layer/employee_course.csv', index=False)

Employee Table

In [144]:
employee_mart_df = pd.read_csv('Staging_Layer/employee.csv')
employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                500 non-null    int64 
 1   empID             500 non-null    object
 2   name              500 non-null    object
 3   designation       500 non-null    object
 4   password          500 non-null    object
 5   email             500 non-null    object
 6   mobile            500 non-null    int64 
 7   performance_rate  500 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 31.4+ KB


In [145]:
# Dropping performance_rate column 
employee_mart_df.drop('performance_rate', axis=1, inplace=True)

# Calculating mean of completion_rates for each employee
mean_completion_rate = employee_course_mart_df.groupby('empID')['completion_rate'].mean().reset_index()

# Rename the completion_rate column to performance_rate for clarity
mean_completion_rate.rename(columns={'completion_rate': 'performance_rate'}, inplace=True)

# Step 2: Merge the mean completion rate back to the employee mart DataFrame
merged_df = employee_mart_df.merge(mean_completion_rate, on='empID', how='left')

# Assign the performance_rate from the merged DataFrame back to employee_mart_df
employee_mart_df['performance_rate'] = merged_df['performance_rate']

employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    int64  
 1   empID             500 non-null    object 
 2   name              500 non-null    object 
 3   designation       500 non-null    object 
 4   password          500 non-null    object 
 5   email             500 non-null    object 
 6   mobile            500 non-null    int64  
 7   performance_rate  494 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 31.4+ KB


In [146]:
employee_mart_df.dropna(subset=['empID'], inplace=True)
employee_mart_df['name'].fillna('Unknown',inplace=True)
employee_mart_df['designation'].fillna('Not Known', inplace=True)
employee_mart_df['email'].fillna("Not Known", inplace=True)
employee_mart_df['password'].fillna(employee_mart_df['empID'],inplace=True)
employee_mart_df['mobile'].fillna(0,inplace=True)
employee_mart_df['performance_rate'].fillna(0,inplace=True)
employee_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    int64  
 1   empID             500 non-null    object 
 2   name              500 non-null    object 
 3   designation       500 non-null    object 
 4   password          500 non-null    object 
 5   email             500 non-null    object 
 6   mobile            500 non-null    int64  
 7   performance_rate  500 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 31.4+ KB


C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\667963262.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  employee_mart_df['name'].fillna('Unknown',inplace=True)
C:\Users\AkhilKumar\AppData\Local\Temp\ipykernel_2588\667963262.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [147]:
# Dropping duplicates
employee_mart_df=employee_mart_df.drop_duplicates()

# Rounding performance_rate to two decimals
employee_mart_df['performance_rate'] = employee_mart_df['performance_rate'].round(2)

In [148]:
# Moving data into mart layer
employee_mart_df.to_csv('Mart_Layer/employee.csv')

----------------------------------------------------------------------------------Report Layer----------------------------------------------------------------------------------

In [149]:
fact_df = employee_course_mart_df.merge(course_mart_df, on='courseID', how='left').merge(employee_mart_df, on='empID', how='left')
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empID             2000 non-null   object 
 1   courseID          2000 non-null   object 
 2   completion_rate   2000 non-null   float64
 3   id_x              2000 non-null   int64  
 4   name_x            2000 non-null   object 
 5   duration          2000 non-null   object 
 6   difficulty_level  2000 non-null   object 
 7   learning_path     2000 non-null   object 
 8   duration_weeks    2000 non-null   int64  
 9   id_y              2000 non-null   int64  
 10  name_y            2000 non-null   object 
 11  designation       2000 non-null   object 
 12  password          2000 non-null   object 
 13  email             2000 non-null   object 
 14  mobile            2000 non-null   int64  
 15  performance_rate  2000 non-null   float64
dtypes: float64(2), int64(4), object(10)
memory

In [150]:
fact_df = fact_df[['empID','designation','courseID','name_x','duration_weeks','difficulty_level','completion_rate','learning_path']]
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empID             2000 non-null   object 
 1   designation       2000 non-null   object 
 2   courseID          2000 non-null   object 
 3   name_x            2000 non-null   object 
 4   duration_weeks    2000 non-null   int64  
 5   difficulty_level  2000 non-null   object 
 6   completion_rate   2000 non-null   float64
 7   learning_path     2000 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 125.1+ KB


In [151]:
fact_df.rename(columns={'name_x':'course_name'}, inplace=True)
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empID             2000 non-null   object 
 1   designation       2000 non-null   object 
 2   courseID          2000 non-null   object 
 3   course_name       2000 non-null   object 
 4   duration_weeks    2000 non-null   int64  
 5   difficulty_level  2000 non-null   object 
 6   completion_rate   2000 non-null   float64
 7   learning_path     2000 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 125.1+ KB


In [156]:
fact_df.to_csv('Report_Layer/fact.csv', index=False)
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empID             2000 non-null   object 
 1   designation       2000 non-null   object 
 2   courseID          2000 non-null   object 
 3   course_name       2000 non-null   object 
 4   duration_weeks    2000 non-null   int64  
 5   difficulty_level  2000 non-null   object 
 6   completion_rate   2000 non-null   float64
 7   learning_path     2000 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 125.1+ KB


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [153]:
course_mart_df['learning_path'].unique()

array(['Data Science', 'Cloud Computing', 'Machine Learning',
       'Web Development', 'Cybersecurity'], dtype=object)